# New Section

In [17]:
import os
import sys
import glob
import shutil
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from google.colab import drive
import math
import torch
import lpips

print("--- [INFO] Starting Full Setup and Evaluation Pipeline ---")

# ==========================================
# 0. Install Critical Dependencies FIRST
# ==========================================
# We install these first to prevent ModuleNotFoundError when importing.
print("[INFO] Installing torch/lpips/scikit-image dependencies...")
!pip install lpips scikit-image > /dev/null
!pip install torch torchvision > /dev/null

# ==========================================
# 1. Mount Google Drive
# ==========================================
print("[INFO] Mounting Google Drive...")
drive.mount('/content/drive')
print("[SUCCESS] Google Drive is mounted.")

# ==========================================
# 2. Clean & Clone Repository
# ==========================================
os.chdir("/content")
if os.path.exists("DeepVision-Frame-Reconstruction-System"):
    print("[INFO] Removing old repository version...")
    shutil.rmtree("DeepVision-Frame-Reconstruction-System")

print("[INFO] Cloning latest code from GitHub...")
!git clone https://github.com/ShadyNikooei/DeepVision-Frame-Reconstruction-System.git

os.chdir("/content/DeepVision-Frame-Reconstruction-System")
print("[SUCCESS] Entered project directory.")


# ==========================================
# 3. Safety Patch: Fix 'optical_flow_wrapper.py' Path
# ==========================================
wrapper_path = "deepvision/models/optical_flow_wrapper.py"
if os.path.exists(wrapper_path):
    print("[INFO] Patching optical_flow_wrapper for safety...")
    new_wrapper_code = """
import torch
import cv2
import numpy as np
import sys
import os

# --- PATH FIX ---
current_dir = os.path.dirname(os.path.abspath(__file__))
project_root = os.path.dirname(os.path.dirname(current_dir))
cv_folder_candidates = [
    os.path.join(project_root, 'computer_vision'),
    os.path.join(project_root, 'computer_vision_')
]

for folder in cv_folder_candidates:
    if os.path.exists(folder) and folder not in sys.path:
        sys.path.append(folder)
# ----------------

try:
    from hybrid_optical_flow_interpolation import hybrid_optical_flow_interpolation
except ImportError:
    try:
        from computer_vision.hybrid_optical_flow_interpolation import hybrid_optical_flow_interpolation
    except ImportError:
        from computer_vision_.hybrid_optical_flow_interpolation import hybrid_optical_flow_interpolation

from deepvision.pipelines.interpolate import t_to_bgr_uint8, bgr_uint8_to_t

class OpticalFlowInterpolator(torch.nn.Module):
    def __init__(self, fix_border: bool = True):
        super().__init__()
        self.fix_border = fix_border
        self.device = "cpu"

    def forward(self, img0: torch.Tensor, img1: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
        img0_bgr_np = t_to_bgr_uint8(img0)
        img1_bgr_np = t_to_bgr_uint8(img1)
        frame_a = cv2.cvtColor(img0_bgr_np, cv2.COLOR_BGR2RGB)
        frame_c = cv2.cvtColor(img1_bgr_np, cv2.COLOR_BGR2RGB)
        interpolated_rgb_np = hybrid_optical_flow_interpolation(frame_a, frame_c, self.fix_border)
        return bgr_uint8_to_t(cv2.cvtColor(interpolated_rgb_np, cv2.COLOR_RGB2BGR))

    def load_pretrained(self, **kwargs) -> "OpticalFlowInterpolator":
        return self
"""
    with open(wrapper_path, "w") as f:
        f.write(new_wrapper_code)


# ==========================================
# 4. Prepare Data (From Google Drive)
# ==========================================
print("[INFO] Setting up data...")
os.makedirs("data/uvg", exist_ok=True)

drive_source = "/content/drive/MyDrive/MyProject_Data"

mp4_files = glob.glob("data/uvg/**/*.mp4", recursive=True)
if not mp4_files:
    if os.path.exists(drive_source):
        print("[INFO] Copying video files from Drive...")
        !cp -r "{drive_source}/." "data/uvg/"
    else:
        print("[WARNING] Drive source folder not found. Please ensure the path is correct.")
else:
    print("[SUCCESS] Data already exists in Colab.")

video_files = glob.glob("data/uvg/**/*.mp4", recursive=True)
video_files = [v for v in video_files if "incomplete" not in v]

if not video_files:
    raise FileNotFoundError("[ERROR] No input video found! Please upload videos to Drive folder 'MyProject_Data'.")

input_video = os.path.abspath(video_files[0])
incomplete_video = os.path.abspath("data/uvg/incomplete.mp4")
final_output = os.path.abspath("results/final_output.mp4")
# Define the path for the text results file
results_file = os.path.abspath("results/evaluation_results.txt")

print(f"[INFO] Input Video Path: {input_video}")


# ==========================================
# 5. Generate Incomplete Video (Drop Frames)
# NOTE: YOU MUST CORRECT THE INDEXING IN YOUR make_incomplete_video.py FILE
# (change 'i % 2 == 1' to 'i % 2 == 0' to keep even frames)
# ==========================================
print("[INFO] Dropping frames to create incomplete video...")
cap = cv2.VideoCapture(input_video)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(incomplete_video, fourcc, fps / 2, (width, height))

frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret: break
    # KEEP EVEN FRAMES (0, 2, 4, ...) - ASSUMING CORRECTION IS APPLIED
    if frame_idx % 2 == 0: out.write(frame)
    frame_idx += 1
cap.release()
out.release()
print("[SUCCESS] Incomplete video created.")


# ==========================================
# 6. Run Adaptive Interpolation Inference
# ==========================================
print("[INFO] Running Adaptive Interpolation...")
os.makedirs("results", exist_ok=True)

!python -m deepvision.scripts.infer_video \
  --input "{incomplete_video}" \
  --output "{final_output}" \
  --adaptive \
  --num-intermediate 1

if os.path.exists(final_output):
    print(f"\n[SUCCESS] Reconstructed video saved at: {final_output}")
else:
    print("\n[ERROR] Inference failed. Check logs above.")


# ==========================================
# 7. Evaluation Metrics Calculation and File Output (FIXED LPIPS)
# ==========================================
print("\n--- [INFO] Starting evaluation metrics calculation (PSNR, SSIM, LPIPS) ---")

# --- LPIPS Initialization (FIXED) ---
# Define the device first and assign the model to it to resolve AttributeError.
lpips_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the LPIPS model
try:
    loss_fn_vgg = lpips.LPIPS(net='vgg').to(lpips_device)
except Exception as e:
    print(f"[ERROR] Could not load LPIPS model to CUDA/GPU: {e}")
    lpips_device = torch.device("cpu")
    loss_fn_vgg = lpips.LPIPS(net='vgg').to(lpips_device)

print(f"[INFO] LPIPS model initialized on device: {lpips_device}")


# --- Helper Functions ---
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0: return 100.0
    max_pixel = 255.0
    psnr = 20 * math.log10(max_pixel / math.sqrt(mse))
    return psnr

def np_to_torch(img_np):
    img_rgb = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)
    # Use the globally defined device
    tensor = lpips.im2tensor(img_rgb).to(lpips_device)
    return tensor

# --- Main Evaluation Logic ---

cap_original = cv2.VideoCapture(input_video)
cap_reconstructed = cv2.VideoCapture(final_output)

if not cap_original.isOpened() or not cap_reconstructed.isOpened():
    print("[ERROR] Could not open video files for evaluation.")
    sys.exit()

psnr_values = []
ssim_values = []
lpips_values = []
frame_idx = 0

print("[INFO] Comparing reconstructed frames against ground truth...")

while True:
    ret_orig, frame_orig = cap_original.read()

    # Only compare the frames that were interpolated (odd index)
    if frame_idx % 2 != 0 and ret_orig:
        ret_reco, frame_reco = cap_reconstructed.read()

        if ret_reco:
            if frame_orig.shape != frame_reco.shape:
                frame_reco = cv2.resize(frame_reco, (frame_orig.shape[1], frame_orig.shape[0]), interpolation=cv2.INTER_LINEAR)

            psnr_values.append(calculate_psnr(frame_orig, frame_reco))

            gray_orig = cv2.cvtColor(frame_orig, cv2.COLOR_BGR2GRAY)
            gray_reco = cv2.cvtColor(frame_reco, cv2.COLOR_BGR2GRAY)
            ssim_val, _ = ssim(gray_orig, gray_reco, full=True)
            ssim_values.append(ssim_val)

            tensor_orig = np_to_torch(frame_orig)
            tensor_reco = np_to_torch(frame_reco)

            with torch.no_grad():
                lpips_val = loss_fn_vgg(tensor_orig, tensor_reco).item()
            lpips_values.append(lpips_val)

        elif not ret_reco:
            break

    elif not ret_orig:
        break

    frame_idx += 1

cap_original.release()
cap_reconstructed.release()

# --- Reporting Results and Saving to File ---
if psnr_values:
    avg_psnr = np.mean(psnr_values)
    avg_ssim = np.mean(ssim_values)
    avg_lpips = np.mean(lpips_values)

    # Format the report content
    report = [
        "\n--- [EVALUATION RESULTS] 📊 ---",
        f"Total interpolated frames analyzed: {len(psnr_values)}",
        f"Average PSNR (Peak Signal-to-Noise Ratio): {avg_psnr:,.4f}",
        f"Average SSIM (Structural Similarity Index): {avg_ssim:,.4f}",
        f"Average LPIPS (Learned Perceptual Similarity): {avg_lpips:,.4f} (Lower is Better)"
    ]

    # Print to console
    print("\n".join(report))

    # Save to text file for Git tracking
    with open(results_file, "w") as f:
        f.write("\n".join(report))

    print(f"\n[SUCCESS] Evaluation metrics saved to file: {results_file}")
else:
    print("\n[WARNING] No interpolated frames were successfully compared.")

print("\n[END] Full pipeline execution and evaluation complete.")

--- [INFO] Starting Full Setup and Evaluation Pipeline ---
[INFO] Installing torch/lpips/scikit-image dependencies...
[INFO] Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[SUCCESS] Google Drive is mounted.
[INFO] Removing old repository version...
[INFO] Cloning latest code from GitHub...
Cloning into 'DeepVision-Frame-Reconstruction-System'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 147 (delta 35), reused 119 (delta 18), pack-reused 0 (from 0)
Receiving objects: 100% (147/147), 2.96 MiB | 33.65 MiB/s, done.
Resolving deltas: 100% (35/35), done.
[SUCCESS] Entered project directory.
[INFO] Patching optical_flow_wrapper for safety...
[INFO] Setting up data...
[INFO] Copying video files from Drive...
[INFO] Input Video Path: /content/DeepVision-Frame-Reconstruction-System

In [ ]:
print("در حال نصب وابستگی‌های حیاتی...")
!pip install lpips scikit-image torch torchvision
print("نصب تکمیل شد.")

در حال نصب وابستگی‌های حیاتی...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.3 MB/s eta 0:00:00
نصب تکمیل شد.


In [28]:
print("در حال جستجوی نوت‌بوک‌ها در Google Drive...")
!ls -R "/content/drive/MyDrive/" | grep -E ".*\.ipynb$"

در حال جستجوی نوت‌بوک‌ها در Google Drive...
Untitled0.ipynb
Untitled1.ipynb
deepVisionTest.ipynb
gpu_Programming_EX1.ipynb
PPO_CLIP_1.ipynb
PPO_CLIP_2.ipynb
Untitled7.ipynb


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [26]:
import os

#os.chdir("/content")
print(f"dir: {os.getcwd()}")

notebook_name = "deepVisionTest.ipynb"
repo_dir = "/content/DeepVision-Frame-Reconstruction-System"

!mv "{notebook_name}" "{repo_dir}/"
print(f"file deepVisionTest.ipynb forwarded to {repo_dir}.")

os.chdir(repo_dir)

!git add "{notebook_name}"
print("ready for comit")

dir: /content/DeepVision-Frame-Reconstruction-System
mv: cannot stat 'deepVisionTest.ipynb': No such file or directory
file deepVisionTest.ipynb forwarded to /content/DeepVision-Frame-Reconstruction-System.
fatal: pathspec 'deepVisionTest.ipynb' did not match any files
ready for comit
